In [3]:
# Run cell to import libraries 
import geopandas as gpd 
import pandas as pd
import matplotlib.pyplot as plt
import contextily 
import mapclassify 
import folium
import aiohttp
import fsspec

# This loads geodataframe contaning county geometry shapes
c = "https://github.com/babdelfa/gis/blob/main/counties_geometry.zip?raw=true"
import fsspec
with fsspec.open(c) as file:
    county_shapes = gpd.read_file(file)

# This loads the most recent covid19 data from Johns Hopkins University's Github
url_cases = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
df_cases = pd.read_csv(url_cases) 
url_deaths = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df_deaths = pd.read_csv(url_deaths) 


In [4]:
# Project code & output

# Subsetting and Melting cases dataframe: We must first subset the dataframe to only
# include the columns that we need to use. Then, we use the pd.melt() function in 
# order to include the variable 'dates' and the values 'cases'.

df_cases = df_cases[['Province_State', 'Admin2', 'FIPS', '3/9/23']]

df_cases_melt = pd.melt(df_cases, id_vars = ["Province_State", 'Admin2', "FIPS"],
                       var_name = "dates",
                       value_name = "cases")
df_cases_melt['dates'] = pd.to_datetime(df_cases_melt['dates'])

# Subsetting and Melting deaths dataframe: We must first subset the dataframe to only 
# include the columns that we need to use. Then, we use the pd.melt() function in order
# to include the variable 'dates' and the values 'deaths'.

df_deaths = df_deaths[['Province_State', 'Admin2', 'FIPS', '3/9/23']]

df_deaths_melt = pd.melt(df_deaths, id_vars = ["Province_State", 'Admin2', "FIPS"],
                       var_name = "dates",
                       value_name = "deaths")

# Set dates to datetime object.
df_deaths_melt['dates'] = pd.to_datetime(df_deaths_melt['dates'])



# Merging the melted cases and melted deaths dataframes: We can then use the melted
# dfs above to merge them into one df using pd.merge(). This is done by joining them
# together based on their related columns (as shown below).

df_us_merged = pd.merge(df_cases_melt, df_deaths_melt, how = 'left',
                       on = ['Admin2', 'Province_State', 'FIPS', 'dates'])


# Rename columns in our merged dataframe: We can use .rename to rename our columns
# for more clarity and better understanding of the data.

df_us_merged.rename(columns = {"Admin2":"County", "Province_State":"State", "cases":"Total Cases", 
                               "deaths":"Total Deaths"},
                    inplace = True)

# Variables indicating totals for USA cases and USA deaths:

total_usa_cases = df_us_merged['Total Cases'].sum()
total_usa_deaths = df_us_merged["Total Deaths"].sum()

# Projected Output: In this frenzy of f-strings and print statements, the user is 
# asked to input their name, which in turn greets them and presents information 
# about the USA totals shown above. Then, the user is asked to input a state.

name = input("Enter your name: ")


print(f"Hello {name}\n")
print("COVID-19 Project")
print(f"By: {name}")
print("General data information:")
print(f"\t Topic: COVID-19 reported cases & death")
print(f"\t Data source: Johns Hopkins University")
print(f"\t Time period: {df_cases.columns[3]}\n")
print("USA OVERVIEW")
print(f"\t - Total reported cases: {total_usa_cases}")
print(f"\t - Total reported deaths: {total_usa_deaths}\n\n")
print(f"{name}, enter a state name for more COVID-19 information: ")

input_state = input(f">>>>>>Enter state: ")

# Variables indicating totals for State cases and State deaths:

total_state_cases = df_us_merged.loc[df_us_merged['State'] == input_state, 
                                     'Total Cases'].sum()

total_state_deaths = df_us_merged.loc[df_us_merged['State'] == input_state, 
                                      'Total Deaths'].sum()
                   
# This 'if' statement is meant to decipher whether or not a user has inputted
# a valid state. Essentially, it is saying if the user has inputted a state that
# is also in our merged df, provide the information for that state.

if input_state in df_us_merged['State'].unique():
    print(f"Found information for {input_state}.\n\n")
    print(f"{input_state} OVERVIEW")
    print(f"\t - Total reported cases: {total_state_cases}")
    print(f"\t - Total reported deaths: {total_state_deaths}\n")
    print(f"Below is an interactive map of COVID-19 cases and deaths for ")
    print(f"{input_state} counties.")
    
# The 'chosen_state' and 'chosen_state_df' dfs are used to merge the inputted
# state from the user. The final line of our code uses .explore to 
# create a choropleth of the given state.

chosen_state = df_us_merged[df_us_merged['State'] == input_state].copy()
chosen_state_df = pd.merge(county_shapes, chosen_state, left_on = "FIPS_BEA", 
                           right_on = "FIPS")
chosen_state_df.drop(['dates'], axis=1, inplace=True)
chosen_state_df.explore(column = 'Total Deaths', cmap = 'Set2', legend = True, 
                        scheme = 'NaturalBreaks', 
                     tooltip=[('County'), ('Total Cases'), ('Total Deaths')])


Enter your name: Johnathan
Hello Johnathan

COVID-19 Project
By: Johnathan
General data information:
	 Topic: COVID-19 reported cases & death
	 Data source: Johns Hopkins University
	 Time period: 3/9/23

USA OVERVIEW
	 - Total reported cases: 103802702
	 - Total reported deaths: 1123836


Johnathan, enter a state name for more COVID-19 information: 
>>>>>>Enter state: Maine
Found information for Maine.


Maine OVERVIEW
	 - Total reported cases: 318130
	 - Total reported deaths: 2928

Below is an interactive map of COVID-19 cases and deaths for 
Maine counties.
